# CFG

In [1]:
# ====================================================
# CFG
# ====================================================
class CFG:
    num_workers=4
    path="/kaggle/input/pppm-deberta-v3-large-baseline-w-w-b-t/"
#     path="../input/pppm-bert-for-patents-baseline-w-w-b-train/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
#     model="anferico/bert-for-patents"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=60 # CHANGED: 133>60
    seed=42
    n_fold=4
    trn_fold=[0]
    
INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'

In [2]:
# attention is all you need 
example1 = "The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data"

# rephrased using chatgpt
example2 = "The current popular models for sequence transduction use complex recurrent or convolutional neural networks in an encoder-decoder configuration, with the best ones utilizing an attention mechanism to connect the encoder and decoder. However, we introduce a new network architecture called the Transformer, which solely uses attention mechanisms and eliminates the need for recurrence and convolutions. Our experiments on two machine translation tasks demonstrate that the Transformer models are not only of higher quality but also more parallelizable and require significantly less training time. For example, our model achieves a BLEU score of 28.4 on the WMT 2014 English-to-German translation task, surpassing the existing best results, even those using ensembles. Additionally, our model achieves a new state-of-the-art BLEU score of 41.8 on the WMT 2014 English-to-French translation task with a single model, after training for only 3.5 days on eight GPUs, which is a small fraction of the training costs of the best models from previous research. Furthermore, we demonstrate that the Transformer can be applied to other tasks, such as English constituency parsing, with great success even with limited training data, showing that it has good generalization capabilities"

# resnet paper
example3 = "Deeper neural networks are more difficult to train. We present a residual learning framework to ease the training of networks that are substantially deeper than those used previously. We explicitly reformulate the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced functions. We provide comprehensive empirical evidence showing that these residual networks are easier to optimize, and can gain accuracy from considerably increased depth. On the ImageNet dataset we evaluate residual nets with a depth of up to 152 layers---8x deeper than VGG nets but still having lower complexity. An ensemble of these residual nets achieves 3.57% error on the ImageNet test set. This result won the 1st place on the ILSVRC 2015 classification task. We also present analysis on CIFAR-10 with 100 and 1000 layers.The depth of representations is of central importance for many visual recognition tasks. Solely due to our extremely deep representations, we obtain a 28% relative improvement on the COCO object detection dataset. Deep residual nets are foundations of our submissions to ILSVRC & COCO 2015 competitions, where we also won the 1st places on the tasks of ImageNet detection, ImageNet localization, COCO detection, and COCO segmentation."

# Library

In [3]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
# import joblib
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import itertools
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


torch.__version__: 1.9.1
Found existing installation: transformers 4.16.2
Uninstalling transformers-4.16.2:
  Successfully uninstalled transformers-4.16.2


Found existing installation: tokenizers 0.11.6
Uninstalling tokenizers-0.11.6:
  Successfully uninstalled tokenizers-0.11.6


Looking in links: ../input/pppm-pip-wheels-dataset
Processing /kaggle/input/pppm-pip-wheels-dataset/transformers-4.16.2-py3-none-any.whl
Processing /kaggle/input/pppm-pip-wheels-dataset/tokenizers-0.11.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl


Looking in links: ../input/pppm-pip-wheels-dataset


tokenizers.__version__: 0.11.6
transformers.__version__: 4.16.2
env: TOKENIZERS_PARALLELISM=true


# Utils

In [4]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# tokenizer

In [5]:
# ====================================================
# tokenizer
# ====================================================
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

# Dataset

In [6]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs

# Model

In [7]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

# inference

In [8]:
# ====================================================
# inference
# ====================================================
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
#     tk0 = tqdm(test_loader, total=len(test_loader))
#     for inputs in tk0:
    for inputs in test_loader:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

# Quora (Inference Pipeline)

In [9]:
pip install -q keybert

Note: you may need to restart the kernel to use updated packages.


In [10]:
from keybert import KeyBERT
kw_model = KeyBERT()

def get_keywords(data, n_top=5, n_words=2):
    return kw_model.extract_keywords(
        data, 
        keyphrase_ngram_range=(1, n_words), 
        # use_mmr=True, nr_candidates=20, # diversity
        top_n=n_top,
        # highlight=True
    )

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [11]:
# !mkdir -p data
# !cp ../input/question-pairs-dataset/questions.csv ./data

In [12]:
# N_SAMPLES = 20000

# quora_df = pd.read_csv('./data/questions.csv').fillna("")
# quora_df = quora_df.head(N_SAMPLES)
# quora_df = quora_df.drop(columns=['qid1', 'qid2', 'is_duplicate'])
# quora_df.head()

quora_df = pd.DataFrame({
    'id': [0, 1, 2],
    'question1': [example1, example1, example2],
    'question2': [example2, example3, example3]
})
quora_df.head()

,id,question1,question2
0,0,The dominant sequence transduction models are ...,The current popular models for sequence transd...
1,1,The dominant sequence transduction models are ...,Deeper neural networks are more difficult to t...
2,2,The current popular models for sequence transd...,Deeper neural networks are more difficult to t...


In [13]:
pip install -q -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [14]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


def get_score_sbert(sent1, sent2):
    embedding_1= model.encode(sent1, convert_to_tensor=True, show_progress_bar=False)
    embedding_2 = model.encode(sent2, convert_to_tensor=True, show_progress_bar=False)
    
    return util.pytorch_cos_sim(embedding_1, embedding_2)[0].item()

get_score_sbert("I'm happy", "I'm full of happiness")

0.6002566814422607

In [15]:
%%time
quora_df['label'] = quora_df.apply(lambda x: get_score_sbert(x['question1'], x['question2']), axis=1)
quora_df.to_csv('quora_labels.csv', index=False)
quora_df.head()

CPU times: user 41.8 ms, sys: 3.74 ms, total: 45.5 ms
Wall time: 49 ms


,id,question1,question2,label
0,0,The dominant sequence transduction models are ...,The current popular models for sequence transd...,0.988794
1,1,The dominant sequence transduction models are ...,Deeper neural networks are more difficult to t...,0.541782
2,2,The current popular models for sequence transd...,Deeper neural networks are more difficult to t...,0.549304


In [16]:
%%time
# Keyword extraction -> keyphrase pairs dataframe
N_TOP_KEYWORD = 5

test = pd.DataFrame(columns=['anchor', 'target', 'sample_id', 'text'])

for i in tqdm(range(len(quora_df)), total=len(quora_df)):
    sample = quora_df.iloc[i]
    k1list = [key for key, _ in get_keywords(sample.question1, n_top=N_TOP_KEYWORD)]
    k2list = [key for key, _ in get_keywords(sample.question2, n_top=N_TOP_KEYWORD)]
    keyword_pairs = itertools.product(k1list, k2list)
    df = pd.DataFrame(keyword_pairs, columns=['anchor', 'target'])
    df['sample_id'] = sample.id
    df['sample_id'] = df['sample_id'].astype(int)
    test = pd.concat([test, df], ignore_index=True)    

test['text'] = test['anchor'] + '[SEP]' + test['target']
test.head()

  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 749 ms, sys: 279 ms, total: 1.03 s
Wall time: 574 ms


,anchor,target,sample_id,text
0,translation tasks,translation tasks,0,translation tasks[SEP]translation tasks
1,translation tasks,machine translation,0,translation tasks[SEP]machine translation
2,translation tasks,translation task,0,translation tasks[SEP]translation task
3,translation tasks,recurrent convolutional,0,translation tasks[SEP]recurrent convolutional
4,translation tasks,parsing great,0,translation tasks[SEP]parsing great


In [17]:
%%time
# Keyphrase similarity prediction -> Keyphrase pairs + scores

test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []

for fold in CFG.trn_fold:
    print(fold)
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
#     print(prediction)
    
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()

predictions = np.mean(predictions, axis=0)
test['score'] = predictions
test.head()

0
CPU times: user 7.67 s, sys: 2.57 s, total: 10.2 s
Wall time: 25.1 s


,anchor,target,sample_id,text,score
0,translation tasks,translation tasks,0,translation tasks[SEP]translation tasks,0.999392
1,translation tasks,machine translation,0,translation tasks[SEP]machine translation,0.415920
2,translation tasks,translation task,0,translation tasks[SEP]translation task,0.999474
3,translation tasks,recurrent convolutional,0,translation tasks[SEP]recurrent convolutional,0.240311
4,translation tasks,parsing great,0,translation tasks[SEP]parsing great,0.059695


In [18]:
test.to_csv('/kaggle/working/keyphrase_scores.csv', index=False)

In [19]:
# Keyphrase pairs + scores -> Question pairs + scores
TOP_N_METRIC = 5

results = quora_df
results[f'top_{TOP_N_METRIC}_mean'] = test.groupby('sample_id')['score'].apply(lambda x: x.nlargest(TOP_N_METRIC).mean())
results.to_csv('quora_results.csv', index=False)
abs_diff = (results[f'top_{TOP_N_METRIC}_mean'] - results['label']).abs()
mean_abs_diff = abs_diff.mean()
print(f'Mean difference: {mean_abs_diff:.3f}')
results.head()

Mean difference: 0.167


,id,question1,question2,label,top_5_mean
0,0,The dominant sequence transduction models are ...,The current popular models for sequence transd...,0.988794,0.999426
1,1,The dominant sequence transduction models are ...,Deeper neural networks are more difficult to t...,0.541782,0.302509
2,2,The current popular models for sequence transd...,Deeper neural networks are more difficult to t...,0.549304,0.297327


# Post Process with MLP

In [20]:
from tqdm import tqdm

all_ids = test.sample_id.unique()

inputs = []
labels = []
for sid in tqdm(all_ids):
    scores = test[test.sample_id == sid]['score'].values
    if len(scores) < 25:
        scores = np.pad(scores, (0, 25-len(scores)), 'constant')
    
    inputs.append(scores)
    labels.append(results[results.id == sid].label.values[0])
    
inputs = np.stack(inputs)
labels = np.array(labels)

inputs.shape, labels.shape

100%|██████████| 3/3 [00:00<00:00, 572.34it/s]


((3, 25), (3,))

In [21]:
from sklearn.neural_network import MLPRegressor
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(inputs, labels, random_state=42)

# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [22]:
# %%time
# regr = MLPRegressor(random_state=42, max_iter=1000)
# # regr.fit(X_train, y_train)
# # regr.score(X_test, y_test)

In [23]:
import pickle

# filename = 'regressor_mlp.sav'
# pickle.dump(regr, open(filename, 'wb'))
regr2 = pickle.load(open("/kaggle/input/deberta-pipeline-full/regressor_mlp.sav", 'rb'))

In [24]:
preds = regr2.predict(inputs)

In [25]:
def get_mlp_score(row):
    sid = row['id']
    scores = test[test.sample_id == sid]['score'].values
    if len(scores) < 25:
        scores = np.pad(scores, (0, 25-len(scores)), 'constant')
        
    return regr2.predict([scores])[0]

results[f'mlp'] = results.apply(lambda row: get_mlp_score(row), axis=1)
results.head()

,id,question1,question2,label,top_5_mean,mlp
0,0,The dominant sequence transduction models are ...,The current popular models for sequence transd...,0.988794,0.999426,0.975459
1,1,The dominant sequence transduction models are ...,Deeper neural networks are more difficult to t...,0.541782,0.302509,0.383881
2,2,The current popular models for sequence transd...,Deeper neural networks are more difficult to t...,0.549304,0.297327,0.387407


In [26]:
abs_diff = (results['mlp'] - results['label']).abs()
mean_abs_diff = abs_diff.mean()
print(f'Mean difference: {mean_abs_diff:.3f}')

Mean difference: 0.111


In [27]:
sample = quora_df.iloc[0]

q1 = sample.question1
q2 = sample.question2
print(f"q1: {q1}\nq2: {q2}")

k1, k2 = get_keywords(q1, 5), get_keywords(q2, 5)
k1, k2

q1: The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transf

([('translation tasks', 0.5428),
  ('decoder attention', 0.5383),
  ('machine translation', 0.5098),
  ('translation task', 0.47),
  ('recurrent convolutional', 0.4361)],
 [('translation tasks', 0.5565),
  ('machine translation', 0.5186),
  ('translation task', 0.478),
  ('recurrent convolutional', 0.4316),
  ('parsing great', 0.4148)])

In [28]:
sample = quora_df.iloc[1]

q1 = sample.question1
q2 = sample.question2
print(f"q1: {q1}\nq2: {q2}")

k1, k2 = get_keywords(q1, 5), get_keywords(q2, 5)
k1, k2

q1: The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transf

([('translation tasks', 0.5428),
  ('decoder attention', 0.5383),
  ('machine translation', 0.5098),
  ('translation task', 0.47),
  ('recurrent convolutional', 0.4361)],
 [('depth imagenet', 0.5783),
  ('dataset deep', 0.5542),
  ('deep residual', 0.5535),
  ('residual networks', 0.5426),
  ('deeper neural', 0.534)])

In [29]:
sample = quora_df.iloc[2]

q1 = sample.question1
q2 = sample.question2
print(f"q1: {q1}\nq2: {q2}")

k1, k2 = get_keywords(q1, 5), get_keywords(q2, 5)
k1, k2

q1: The current popular models for sequence transduction use complex recurrent or convolutional neural networks in an encoder-decoder configuration, with the best ones utilizing an attention mechanism to connect the encoder and decoder. However, we introduce a new network architecture called the Transformer, which solely uses attention mechanisms and eliminates the need for recurrence and convolutions. Our experiments on two machine translation tasks demonstrate that the Transformer models are not only of higher quality but also more parallelizable and require significantly less training time. For example, our model achieves a BLEU score of 28.4 on the WMT 2014 English-to-German translation task, surpassing the existing best results, even those using ensembles. Additionally, our model achieves a new state-of-the-art BLEU score of 41.8 on the WMT 2014 English-to-French translation task with a single model, after training for only 3.5 days on eight GPUs, which is a small fraction of the 

([('translation tasks', 0.5565),
  ('machine translation', 0.5186),
  ('translation task', 0.478),
  ('recurrent convolutional', 0.4316),
  ('parsing great', 0.4148)],
 [('depth imagenet', 0.5783),
  ('dataset deep', 0.5542),
  ('deep residual', 0.5535),
  ('residual networks', 0.5426),
  ('deeper neural', 0.534)])